In [57]:
# Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import mlflow
import os
import pickle

ruta_actual = os.getcwd()
mlflow.set_tracking_uri(f"file:///{ruta_actual}/mlruns")
mlflow.set_experiment("Modelos de Recomendación - Comparativa")

# Cargamos cada archivo CSV en un DataFrame de pandas para poder manipularlos como tablas de datos
df_events = pd.read_csv("../databases/events.csv")
df_order_items = pd.read_csv("../databases/order_items.csv")
df_orders = pd.read_csv("../databases/orders.csv")
df_products = pd.read_csv("../databases/products.csv")
df_reviews = pd.read_csv("../databases/reviews.csv")
df_users = pd.read_csv("../databases/users.csv")

In [58]:
from sklearn.model_selection import train_test_split

# Se organiza los productos por ordenes
df_interacciones = df_order_items[['order_id', 'product_id']].copy()

# Se divide 80 train / 20 test
# Es mejor dividir por order_id para no separar una misma orden en dos
ordenes_unicas = df_interacciones['order_id'].unique()
train_ids, test_ids = train_test_split(ordenes_unicas, test_size=0.2, random_state=42)

train_data = df_interacciones[df_interacciones['order_id'].isin(train_ids)]
test_data = df_interacciones[df_interacciones['order_id'].isin(test_ids)]

print(f"✅ Datos listos. Ordenes de entrenamiento: {len(train_ids)} | Ordenes de prueba: {len(test_ids)}")

✅ Datos listos. Ordenes de entrenamiento: 16000 | Ordenes de prueba: 4000


In [59]:
# Unimos la tabla consigo misma usando order_id para encontrar parejas
print("Generando pares de productos... (esto puede tardar unos segundos)")
df_pares = pd.merge(train_data, train_data, on='order_id')

# Filtramos para no contar un producto consigo mismo (ej: leche con leche)
df_pares = df_pares[df_pares['product_id_x'] != df_pares['product_id_y']]

# Contamos la frecuencia de cada pareja
matriz_cooc = df_pares.groupby(['product_id_x', 'product_id_y']).size().reset_index(name='frecuencia')

# Ordenamos por los más frecuentes para facilitar la búsqueda
matriz_cooc = matriz_cooc.sort_values(['product_id_x', 'frecuencia'], ascending=[True, False])

print(f"✅ Matriz creada. Se encontraron {len(matriz_cooc)} relaciones únicas entre productos.")

Generando pares de productos... (esto puede tardar unos segundos)
✅ Matriz creada. Se encontraron 79118 relaciones únicas entre productos.


In [60]:
def recomendar_cc_eval(product_id, df_matriz, df_products, top_n=3):
    # categoria del producto seleccionado
    row = df_products[df_products['product_id'] == product_id]
    if row.empty:
        return []

    cat_seed = row.iloc[0]['Category']

    candidatos = df_matriz[df_matriz['product_id_x'] == product_id]
    if candidatos.empty:
        return []

    recs = candidatos.merge(
        df_products[['product_id', 'Category']],
        left_on='product_id_y',
        right_on='product_id',
        how='left'
    )

    recs = recs[
        (recs['Category'] == cat_seed) &
        (recs['product_id_y'] != product_id)
    ]

    return recs['product_id_y'].head(top_n).tolist()


In [61]:
def recomendar_cc_eval(product_id, df_matriz, df_products, top_n=3):
    # Producto semilla
    row = df_products[df_products['product_id'] == product_id]
    if row.empty:
        return []
    cat_seed = row.iloc[0]['Category']
    subcat_seed = row.iloc[0]['SubCategory']
    candidatos = df_matriz[df_matriz['product_id_x'] == product_id]
    if candidatos.empty:
        return []
    recs = candidatos.merge(
        df_products[['product_id', 'Category', 'SubCategory']],
        left_on='product_id_y',
        right_on='product_id',
        how='left'
    )
    recs = recs[recs['product_id_y'] != product_id]
    recs = recs.sort_values('frecuencia', ascending=False)
    recomendaciones = []
    nivel_1 = (
        recs[
            (recs['Category'] == cat_seed) &
            (recs['SubCategory'] == subcat_seed)
        ]
        .drop_duplicates(subset='product_id_y')
    )
    recomendaciones.extend(nivel_1['product_id_y'].tolist())

    if len(recomendaciones) < top_n:
        nivel_2 = (
            recs[
                (recs['Category'] == cat_seed) &
                (recs['SubCategory'] != subcat_seed)
            ]
            .drop_duplicates(subset='product_id_y')
        )
        nivel_2 = nivel_2[
            ~nivel_2['product_id_y'].isin(recomendaciones)
        ]
        faltantes = top_n - len(recomendaciones)
        recomendaciones.extend(
            nivel_2['product_id_y'].head(faltantes).tolist()
        )
    if len(recomendaciones) < top_n:
        nivel_3 = (
            recs[
                recs['Category'] != cat_seed
            ]
            .drop_duplicates(subset='product_id_y')
        )
        nivel_3 = nivel_3[
            ~nivel_3['product_id_y'].isin(recomendaciones)
        ]
        faltantes = top_n - len(recomendaciones)
        recomendaciones.extend(
            nivel_3['product_id_y'].head(faltantes).tolist()
        )
    return recomendaciones[:top_n]


In [62]:
# class ModeloCoocurrencia:
#     def __init__(self, df_matriz, df_products):
#         self.df_matriz = df_matriz
#         self.df_products = df_products

#     def recomendar(self, product_id, top_n=3):
#         return recomendar_cc_eval(
#             product_id,
#             self.df_matriz,
#             self.df_products,
#             top_n
#         )

In [63]:
# # 1. Definir la lógica de rutas (Idéntica a la que ya usas)
# ruta_actual = os.getcwd()

# if os.path.exists(os.path.join(ruta_actual, "modelos_entrenados")):
#     ruta_modelos = os.path.join(ruta_actual, "modelos_entrenados")
# else:
#     ruta_modelos = os.path.join(os.path.dirname(ruta_actual), "modelos_entrenados")

# if not os.path.exists(ruta_modelos):
#     os.makedirs(ruta_modelos)

# # 2. Nombre del archivo para el modelo CC
# archivo_salida_cc = os.path.join(ruta_modelos, "modelo_recomendacion_cc.pkl")

# # 3. Guardar AMBOS DataFrames en un solo archivo
# # Los guardamos como una tupla: (matriz, productos)
# modelo_cc = ModeloCoocurrencia(matriz_cooc, df_products)

# with open(archivo_salida_cc, 'wb') as f:
#     pickle.dump(modelo_cc, f)


# print(f"💾 Modelo CC guardado exitosamente en:\n{archivo_salida_cc}")

In [64]:
def evaluar_coocurrencia(df_test, df_matriz, df_products, k):
    ordenes = df_test.groupby('order_id')['product_id'].apply(list)

    precisions, recalls = [], []
    mrrs, maps, ndcgs = [], [], []

    for items in ordenes:
        if len(items) < 2:
            continue

        for i in range(len(items)):
            seed = items[i]
            objetivos = set(items[:i] + items[i+1:])

            preds = recomendar_cc_eval(seed, df_matriz, df_products, k)
            if len(preds) == 0:
                continue

            hits = [1 if p in objetivos else 0 for p in preds]
            aciertos = sum(hits)

            # Precision / Recall
            precisions.append(aciertos / k)
            recalls.append(aciertos / len(objetivos))

            # MRR
            rr = 0
            for r, p in enumerate(preds, 1):
                if p in objetivos:
                    rr = 1 / r
                    break
            mrrs.append(rr)

            # MAP
            ap, h = 0, 0
            for idx, p in enumerate(preds):
                if p in objetivos:
                    h += 1
                    ap += h / (idx + 1)
            maps.append(ap / min(len(objetivos), k))

            # NDCG
            dcg, idcg = 0, 0
            for idx, p in enumerate(preds):
                if p in objetivos:
                    dcg += 1 / np.log2(idx + 2)
            for idx in range(min(len(objetivos), k)):
                idcg += 1 / np.log2(idx + 2)
            ndcgs.append(dcg / idcg if idcg > 0 else 0)

    if len(precisions) == 0:
        print("⚠️ No se generaron predicciones evaluables")
        return None

    print("\n" + "="*50)
    print(f"📊 REPORTE FINAL CO-OCURRENCIA (K={k})")
    print("="*50)
    print(f"Precision       | {np.mean(precisions):.4f}")
    print(f"Recall          | {np.mean(recalls):.4f}")
    print(f"F1-Score        | {(2*np.mean(precisions)*np.mean(recalls)/(np.mean(precisions)+np.mean(recalls)+1e-9)):.4f}")
    print("-"*50)
    print(f"MRR             | {np.mean(mrrs):.4f}")
    print(f"MAP             | {np.mean(maps):.4f}")
    print(f"NDCG            | {np.mean(ndcgs):.4f}")
    print("="*50)
    name = f"CC_K{k}"
    with mlflow.start_run(run_name=name):

        # --- Parámetros ---
        mlflow.log_param("modelo", "CC")
        mlflow.log_param("k", k)

        # --- Métricas ---
        mlflow.log_metric("precision", np.mean(precisions))
        mlflow.log_metric("recall", np.mean(recalls))
        mlflow.log_metric("f1", (2*np.mean(precisions)*np.mean(recalls)/(np.mean(precisions)+np.mean(recalls)+1e-9)))
        mlflow.log_metric("mrr", np.mean(mrrs))
        mlflow.log_metric("map", np.mean(maps))
        mlflow.log_metric("ndcg", np.mean(ndcgs))

In [65]:
evaluar_coocurrencia(
    test_data,
    matriz_cooc,
    df_products,
    k=5
)


📊 REPORTE FINAL CO-OCURRENCIA (K=5)
Precision       | 0.0873
Recall          | 0.2176
F1-Score        | 0.1246
--------------------------------------------------
MRR             | 0.2406
MAP             | 0.1678
NDCG            | 0.1993
